In [24]:
import pandas as pd
import numpy as np
import random

In [26]:
np.zeros(5)

array([0., 0., 0., 0., 0.])

In [7]:
a = [[2,3,4], [4, 5, 6]]
def save_all_current_accepted(var, var_name):
    """save all_current_order_dicts, all_current_ikelihoods, 
    and all_current_acceptance_ratios
    """
    x = np.arange(start = 1, stop = len(var) + 1, step = 1)
    df = pd.DataFrame({"iteration": x, var_name: var})
    df = df.set_index('iteration')
    return df 

In [22]:
def save_all_current_participant_stages(var):
    df = pd.DataFrame(var)
    df.index.name = 'iteration'
    df.index = df.index + 1
    
    # # Reorder the columns to place 'iteration' at the start
    # cols = ['iteration'] + [col for col in df.columns if col != 'iteration']
    # df = df[cols]
    return df 

In [12]:
df = save_all_current_accepted(a, "a")
df

,a
iteration,
1,"[2, 3, 4]"
2,"[4, 5, 6]"


In [15]:
df[df.index == 1].iloc[:, 2:]

""
iteration
1


In [23]:
df = save_all_current_participant_stages(a)
df

,0,1,2
iteration,,,
1,2,3,4
2,4,5,6


## Analyze conjugate priors results

In [6]:
a = {}
a['max_likelihood'] = 0
a['max stage'] = [1, 2, 3]
pd.DataFrame([a])

,max_likelihood,max stage
0,0,"[1, 2, 3]"


In [22]:
participant_stages_df = pd.read_csv("logs/conjugate_priors/all_current_participant_stages.csv")
participant_stages_df.head()
participant_stages = participant_stages_df.iloc[-1, 1:]
np.array(participant_stages)

```
array([3, 4, 5, 4, 1, 2, 1, 4, 5, 1, 0, 3, 3, 5, 2, 0, 0, 4, 5, 4, 3, 4,
       1, 0, 2, 5, 2, 4, 2, 0, 3, 0, 2, 4, 4, 5, 1, 5, 2, 2, 2, 5, 4, 1,
       5, 1, 2, 0, 5, 1, 3, 3, 3, 1, 3, 4, 2, 4, 1, 4, 1, 2, 2, 4, 1, 3,
       1, 2, 5, 0, 2, 0, 0, 3, 2, 2, 3, 3, 5, 4, 5, 2, 1, 5, 4, 0, 0, 1,
       2, 4, 2, 2, 2, 1, 0, 3, 3, 0, 3, 4])
```

In [23]:
# current_best_order_dict = {'MMSE': 5, 'ADAS': 4, 'AB': 2, 'P-Tau': 3, 'HIP-FCI': 1}
# real_order_dict = {'MMSE': 4, 'ADAS': 5, 'AB': 2, 'P-Tau': 3, 'HIP-FCI': 1}

In [24]:
# original_data = pd.read_csv('data/participant_data.csv')
# original_data['diseased'] = original_data.apply(lambda row: row.k_j > 0, axis = 1)
# data = original_data.drop(['k_j', 'S_n', 'affected_or_not'], axis = 1)
# theta_phi_kmeans = pd.read_csv("data/estimate_means_stds_kmeans.csv")
# biomarkers = data.biomarker.unique()
# num_biomarkers = len(biomarkers)

In [25]:
# # now data_we_have has S_n column
# data['S_n'] = data.apply(lambda row: current_best_order_dict[row['biomarker']], axis = 1)

# # add kj and affected for the whole dataset based on the initial randomized participant_stages
# data = utils.add_kj_and_affected(data, participant_stages, num_participants)
# # print(data.head())

# # get estimated_theta_phi
# estimated_theta_phi = utils.get_theta_phi_conjugate_priors(biomarkers, data, theta_phi_kmeans=theta_phi_kmeans)

In [26]:
# current_best_likelihood = utils.compute_ln_likelihood_assuming_ordering(
#     current_best_order_dict, data, num_biomarkers, estimated_theta_phi)
# current_best_likelihood

In [27]:
# # now data_we_have has S_n column
# data['S_n'] = data.apply(lambda row: real_order_dict[row['biomarker']], axis = 1)

# # add kj and affected for the whole dataset based on the initial randomized participant_stages
# data = utils.add_kj_and_affected(data, participant_stages, num_participants)
# # print(data.head())

# # get estimated_theta_phi
# estimated_theta_phi = utils.get_theta_phi_conjugate_priors(biomarkers, data, theta_phi_kmeans=theta_phi_kmeans)

In [28]:
# real_likelihood = utils.compute_ln_likelihood_assuming_ordering(
#     current_best_order_dict, data, num_biomarkers, estimated_theta_phi)
# real_likelihood

## new

In [19]:
a = [1, 3, 4, 5, 8, 9, 11, 12, 29, 33]
burn_in = 3
import numpy as np
a

[1, 3, 4, 5, 8, 9, 11, 12, 29, 33]

In [20]:
a_toplot = a[burn_in:]
a_toplot

[5, 8, 9, 11, 12, 29, 33]

In [22]:
np.arange(burn_in+1, len(a)+1, 1)

array([ 4,  5,  6,  7,  8,  9, 10])

In [25]:
a = -np.inf
b = -np.inf 
a == b

True